In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(context='poster', style='whitegrid', palette='viridis')
import joblib

In [3]:
antennas = joblib.load('./intermediate/urban_towers.joblib.gz')

In [4]:
antennas.columns

Index(['tower', 'antenna', 'geometry', 'municipality', 'name'], dtype='object')

In [5]:
481 / antennas.shape[0 * 100], 481 / antennas[antennas.intext].shape[0 * 100]

AttributeError: 'GeoDataFrame' object has no attribute 'intext'

In [ ]:
antennas.shape, antennas.tower.unique().shape

In [ ]:
antennas.plot(markersize=1)

In [ ]:
municipalities = joblib.load('./intermediate/urban_municipalities.joblib.gz')
municipalities.plot()

In [ ]:
municipalities.COD_COMUNA.unique()

In [ ]:
name_map = dict(zip([13101, 13102, 13103
    ,13104,13105,13106,13107,13108,13109,13110,13111,13112,13113,13114,13115,13116,13117,13118,13119,13120,13121,13122,13123
    ,13124,13125,13126,13127,13128,13129,13130,13131,13132,13201,13202,13203,13301,13302,13303,13401,13402,13403,13404,13501,
                     13502, 13503, 13504, 13505, 13601, 13602, 13603, 13604, 13605],
        map(lambda x: x.title(), ['Santiago', 'Cerrillos', 'Cerro Navia', 'Conchalí',
        'El Bosque', 'Estación Central', 'Huechuraba', 'Independencia', 'La Cisterna',
        'La Florida', 'La Granja', 'La Pintana',
        'La Reina','Las Condes','Lo Barnechea','Lo Espejo', 'Lo Prado',
        'Macul','Maipú','Ñuñoa','Pedro Aguirre Cerda','Peñalolén',
        'Providencia','Pudahuel','Quilicura','Quinta Normal','Recoleta','Renca',
        'San Joaquín','San Miguel','San Ramón','Vitacura','Puente Alto','Pirque',
        'San José de Maipo','Colina','Lampa','Tiltil','San Bernardo','Buin',
        'Calera de Tango','Paine','Melipilla','Alhué','Curacaví','María Pinto','San Pedro',
        'Talagante','El Monte','Isla de Maipo','Padre Hurtado','Peñaflor'
        ])))

hdi = pd.read_csv('./intermediate/HDI_2013_Comuna.csv', index_col=False)
# aquí aplicamos la transformación y guardamos el resultado en una nueva columna
hdi['NOM_COM'] = hdi['comuna'].map(lambda x: name_map[x])
hdi.set_index('NOM_COM', inplace=True)
hdi

In [ ]:
tower_hdi = antennas.join(hdi.HDI_2013, on='municipality').groupby('tower').aggregate({'HDI_2013': 'mean'}).join(antennas.drop_duplicates(subset='tower').loc[:,('tower', 'municipality')].set_index('tower'))
tower_hdi.sample(5)

In [ ]:
from shapely.geometry import Point

malls  = pd.read_csv('./input/malls_list.csv')
malls['geometry'] = malls.apply(lambda x: Point(x.lon_x, x.lat_x), axis=1)
malls = gpd.GeoDataFrame(malls, geometry='geometry', crs={'init': 'epsg:4326'})
malls

In [ ]:
malls.set_index('mall_id', inplace=True)

In [ ]:
ax = municipalities.join(hdi.HDI_2013, on='NOM_COM').plot(column='HDI_2013', legend=True, 
                                                                   figsize=(12,12), cmap='viridis',
                                                                   edgecolor='white', linewidth=1.5)
plt.title('HDI of Municipalities in Santiago, Chile')
plt.axis('equal');
plt.savefig('./output/municipalities_hdi.png', dpi=100, bbox_inches='tight')

These files come from another project. Meanwhile, I just copy them here.

In [ ]:
highways = joblib.load('./intermediate/highways.joblib.gz')
railways = joblib.load('./intermediate/railways.joblib.gz')
primary = joblib.load('./intermediate/primary_streets.joblib.gz')

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.markers as mmarkers
#mpatches.CirclePolygon(color='#8da0cb', edgecolor='black')

In [ ]:
plt.figure(figsize=(16,16))

ax = plt.gca()

municipalities.plot(color='#EFEFEF', edgecolor='#EFEFEF', ax=ax)
highways.plot(ax=ax, color='black', alpha=0.95, linewidth=1.5)
primary.plot(ax=ax, color='#66c2a5', alpha=0.95, linewidth=1)

ax.set_title('Highways and Primary Streets (OSM)')
ax.set_axis_off()
ax.set_aspect('equal')
railways[pd.isnull(railways.service)].plot(color='#fc8d62', ax=ax, linewidth=3)

malls.plot(ax=ax, color='#8da0cb', edgecolor='black', zorder=30, linewidth=1.)

handles = []
handles.append(mlines.Line2D([], [], color='black', linewidth=1.5, label='Highways'))
handles.append(mlines.Line2D([], [], color='#66c2a5', linewidth=1, label='Primary Streets'))
handles.append(mlines.Line2D([], [], color='#fc8d62', linewidth=3, label='Metro Network'))
handles.append(mpatches.Circle([0.0, 0.0], radius=4, color='#8da0cb', edgecolor='black', label='Malls'))

plt.legend(handles=handles, loc='lower left')

plt.savefig('./output/santiago_urban_network.png', dpi=100, bbox_inches='tight')

In [ ]:
users = pd.read_csv('input/user_mall.tar.gz').set_index('user_mall.csv')
users.head()

In [ ]:
user_hdi = pd.read_csv('intermediate/userHomeAntenna.csv').set_index('num_from').join(tower_hdi, on='tower_in')
user_hdi.head()

In [ ]:
user_hdi['hdi_quantile'] = pd.qcut(user_hdi.HDI_2013, 5, labels=False)

In [ ]:
print(pd.qcut(user_hdi.HDI_2013, 5).value_counts(sort=False).to_latex())

In [ ]:
df = users.join(user_hdi, how='inner').dropna()
df.sample(5)

In [ ]:
df.index.unique().shape

In [ ]:
df.shape

In [ ]:
df = df.join(malls.description_x, on='mall_id')

In [ ]:
df.columns

In [ ]:
df.sample(3)

In [ ]:
mall_muni = (df.groupby(['description_x', 'municipality']).size()
             .reset_index()
             .pivot(index='description_x', columns='municipality')
             .fillna(0)
             .pipe(lambda x: x.div(x.sum(axis=1), axis=0)))

In [ ]:
mall_muni.head()

In [ ]:
mall_muni.columns = mall_muni.columns.get_level_values(1)

In [ ]:
g = sns.clustermap(mall_muni * 100, metric='correlation', method='ward', linewidth=1, 
                   cmap='magma_r', figsize=(20,11), annot=True)
g.ax_heatmap.set_ylabel('Mall')
g.ax_heatmap.set_xlabel('Comuna (Municipality)')
g.cax.set_title('Percent of Visitors from each Municipality', loc='left')
g.savefig('./output/municipality_mall_matrix.png', dpi=100, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(17,9))
sns.heatmap(g.data2d * 100, linewidth=1, cmap='magma_r', square=True, annot=True,
           cbar_kws={'shrink': 0.25, 'fraction': 0.05}, cbar=False)
plt.xlabel('Comuna (Municipality)')
plt.ylabel('')
#g.ax_heatmap.set_ylabel('Mall')
#g.ax_heatmap.set_xlabel('Comuna (Municipality)')
#plt.savefig('./output/municipality_mall_matrix.png', dpi=100, bbox_inches='tight')

In [ ]:
from scipy.stats import entropy

mall_entropy = entropy(mall_muni.T)

mall_entropy.shape

In [ ]:
mall_entropy = pd.Series(mall_entropy, index=mall_muni.index, name='entropy').sort_values()
mall_entropy

In [ ]:
malls = malls.join(mall_entropy, on='description_x')

In [ ]:
malls = gpd.sjoin(malls, municipalities.loc[:,('geometry', 'NOM_COM')], how='left', op='within')

In [ ]:
malls = malls.join(hdi.HDI_2013, on='NOM_COM')

In [ ]:
malls['NOM_COM'].loc[18.0] = 'Huechuraba'
malls['HDI_2013'].loc[18.0] = 0.712375

In [ ]:
malls

In [ ]:
sns.jointplot('entropy', 'HDI_2013', malls)

In [ ]:
df = df.join(malls.HDI_2013, on='mall_id', rsuffix='_mall')

In [ ]:
df.columns

In [ ]:
df.groupby('mall_id').size()

In [ ]:
df['HDI_2013'].describe()

In [ ]:
df['delta_hdi'] = (df['HDI_2013_mall'] - df['HDI_2013']) / df['number_days']

In [ ]:
delta_hdi_users = df.reset_index().groupby('index').aggregate({'delta_hdi': 'mean', 'HDI_2013': 'mean'})

In [ ]:
g = sns.jointplot('HDI_2013', 'delta_hdi', delta_hdi_users, size=8, space=0.1, joint_kws={'alpha': 0.01, 'marker': '.'})

In [ ]:
delta_hdi_malls = df.groupby('description_x').aggregate({'delta_hdi': 'mean', 'HDI_2013': 'mean'})

In [ ]:
delta_hdi_malls

In [ ]:
g = sns.jointplot('HDI_2013', 'delta_hdi', delta_hdi_malls.dropna(), size=8, space=0.1)

In [ ]:
delta_hdi_muni = df.groupby('municipality').aggregate({'delta_hdi': 'mean'}).join(hdi.HDI_2013)

In [ ]:
sns.set_style('white')
g = sns.jointplot('HDI_2013', 'delta_hdi', delta_hdi_muni, size=8, space=0.1)
g.ax_joint.set_ylabel('HDI (Comuna)')
g.ax_joint.set_xlabel('HDI Diff. of Mall Visits by Residents');
g.savefig('./output/municipality_hdi_differences.png', dpi=100, bbox_inches='tight')

In [ ]:
reg_colors = sns.color_palette('viridis', n_colors=3)

In [ ]:
sns.set_style('whitegrid')
fig, axes = plt.subplots(1, 3, figsize=(17,6), sharey=True)

ax = axes[0]
_d = delta_hdi_users.dropna()
sns.regplot(_d.HDI_2013.values, _d.delta_hdi.values, ax=ax, marker='.', color=reg_colors[0],
            scatter_kws={'alpha': 0.005, 'color': 'grey'})
sns.despine(ax=ax)
ax.set_ylabel('HDI(Comuna of Resid.) - HDI(Visited Mall)')
ax.set_xlabel('(a) HDI(Comuna of Residence)')

ax = axes[1]
_d = delta_hdi_malls.dropna()
sns.regplot(_d.HDI_2013.values, _d.delta_hdi.values, ax=ax,  color=reg_colors[1],
            scatter_kws={'alpha': 0.9, 'color': 'grey'})
sns.despine(ax=ax)
ax.set_xlabel('(b) Average HDI(Mall Location)')

ax = axes[2]
_d = delta_hdi_muni.dropna()
sns.regplot(_d.HDI_2013.values, _d.delta_hdi.values, ax=ax,  color=reg_colors[2],
            scatter_kws={'alpha': 0.9, 'color': 'grey'})
sns.despine(ax=ax)
ax.set_xlabel('(b) Average HDI(Comuna)')

plt.subplots_adjust(wspace=0.1)
plt.savefig('./output/hdi_scatters.png', bbox_inches='tight', dpi=100)

In [ ]:
sns.set_style('white')

In [ ]:
df.sample(3)

N = total individuos
n_t = total individuos que visitaron mall t
N_alpha = total de individuos en cat alpha
n_alpha_t = total de individuos en cat alpha que visitan un mall t
r_beta = (n_beta_t / N_beta) / (n_t / N)
exp = 1 / N_alpha * sum n_alpha * r_beta

In [ ]:
df.shape, df.dropna().shape

In [ ]:
df_nn = df.dropna()
df_nn.sample(5)

In [ ]:
import marble as mb

In [ ]:
city_malls = {}

quantiles = df_nn.hdi_quantile.unique()

for idx, row in malls.iterrows():
    if idx == 18.0:
        continue
    city_malls[idx] = {}
    for cat in quantiles:
        q_name = 'Q{}'.format(int(cat) + 1)
        city_malls[idx][q_name] = df_nn[(df_nn.mall_id == idx) & (df_nn.hdi_quantile == cat)].shape[0]
        
city_malls

In [ ]:
exp = mb.exposure(city_malls)

In [ ]:
records = []

for alpha, exp_beta in exp.items():
    for beta, values in exp_beta.items():
        records.append({'source': alpha, 'target': beta, 'exposure': values[0], 'variance': values[1]})
        
records = pd.DataFrame(records)
records

In [ ]:
# 1 + 2.57σ

In [ ]:
records['std'] = records.variance.pow(0.5)

In [ ]:
records['std'].describe()

In [ ]:
for idx, row in records.iterrows():
    if row.exposure > 1:
        result = row.exposure > (1 + 2.57 * row.get('std'))
    elif row.exposure < 1:
        result = row.exposure < (1 - 2.57 * row.get('std'))
    else:
        result = 'n/a'
    print(row.source, row.target, result)

In [ ]:
sns.heatmap(pd.pivot_table(records, index='source', columns='target', values='exposure'), annot=True, center=1.0,
           cmap='PuOr', linewidth=1)

In [ ]:
x_order = ['Q{}'.format(i) for i in range(1, 6)]
hue_order = x_order
g = sns.factorplot(x='source', y='exposure', data=records, hue='target', hue_order=hue_order,
               aspect=2, size=8, palette='viridis', kind='bar', order=x_order,
                  legend_out=True)
#g.map()
plt.axhline(y=1, ls=":", c=".5")

plt.xlabel('Source HDI Quantile')
plt.ylabel('Exposure(Source HDI Quantile, Target HDI Quantile)')
g.savefig('./output/hdi_exposure.png', dpi=100, bbox_inches='tight')